In [9]:
import pandas as pd
import numpy as np
from selenium import webdriver
import requests
import bs4
from bs4 import BeautifulSoup
import time

In [10]:
qualifying_results = pd.DataFrame()
for year in list(range(2020,2022)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])

In [11]:
qualifying_results.rename(columns = {'Pos': 'grid_pos', 'Driver': 'driver', 'Car': 'constructor', 'Time': 'qualy_time'}, inplace=True)

qualifying_results.drop('No', axis=1, inplace=True)

qualifying_results.head()

,grid_pos,driver,constructor,qualy_time,season,round
0,1,Valtteri Bottas BOT,Mercedes,1:02.939,2020,1
1,2,Max Verstappen VER,Red Bull Racing Honda,1:03.477,2020,1
2,3,Lando Norris NOR,McLaren Renault,1:03.626,2020,1
3,4,Alexander Albon ALB,Red Bull Racing Honda,1:03.868,2020,1
4,5,Lewis Hamilton HAM,Mercedes,1:02.951,2020,1


In [12]:
qualifying_results.to_csv('data/qualifying.csv', index = False)